In [3]:
import os
os.chdir('C:\\Users\competition_data/water_data/*.csv")')

FileNotFoundError: ignored

In [2]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

#GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import tensorflow_addons as tfa
from keras import backend as K
from tensorflow.keras import layers

ModuleNotFoundError: ignored

In [ ]:
rf_list = sorted(glob("rf_data/*.csv"))
rf_list

In [ ]:
w_list = sorted(glob("water_data/*.csv"))
w_list

In [ ]:
#전체 데이터 병합후 저장

for i in range(len(rf_list)):
    pd.concat([pd.read_csv(rf_list[i], index_col = 0),pd.read_csv(w_list[i], index_col = 0)], axis=1).to_csv('full_data/f_list_'+str(i).zfill(2)+'.csv')

In [ ]:
# 병합데이터 불러오기

f_list = sorted(glob("full_data/*.csv")) #
f_list

['full_data\\f_list_00.csv',
 'full_data\\f_list_01.csv',
 'full_data\\f_list_02.csv',
 'full_data\\f_list_03.csv',
 'full_data\\f_list_04.csv',
 'full_data\\f_list_05.csv',
 'full_data\\f_list_06.csv',
 'full_data\\f_list_07.csv',
 'full_data\\f_list_08.csv',
 'full_data\\f_list_09.csv',
 'full_data\\f_list_10.csv']

In [ ]:
pd.read_csv(f_list[0]).shape

(26496, 18)

In [ ]:
pd.read_csv(f_list[0]).head(4)

,ymdhm,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,0.0,0.0,0.0,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,0.0,0.0,0.0,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,0.0,0.0,0.0,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,0.0,0.0,0.0,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [ ]:
#window_size 200으로 묶어 train dataset 만들기

train_data = []
train_label = []
num = 0

window = 30

for i in f_list[:-1]:
    
    tmp = pd.read_csv(i)
    
    # 10분전, 20분전 데이터와의 차이값을 추가 변수로 활용
    
    tmp[["wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]].reset_index(drop=True) - pd.concat([tmp[:1][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]],
                                                                                                             tmp[:-1][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)
    tmp[["wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]].reset_index(drop=True) - pd.concat([tmp[:2][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]],
                                                                                                             tmp[:-2][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)

    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)

    for j in tqdm(range(len(tmp)-window)):
        train_data.append(np.array(tmp.loc[j:j + window-1,
                                           ['rf_10184100','rf_10184110','rf_10184140',"swl", "inf", "sfw", "ecpc",
                                            "tototf", "tide_level","fw_1018662", "fw_1018680","fw_1018683", "fw_1019630",
                                            "wl_1018662", "wl_1018680", "wl_1018683","wl_1019630",
                                            "wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1",
                                           "wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + window, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26466/26466 [00:19<00:00, 1373.29it/s]


In [ ]:
# Train dataset을 계속 사용할 수 있도록 저장

train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

np.save("train_data2.npy",train_data)
np.save("train_label2.npy",train_label)

(264660, 30, 25)
(264660, 4)


In [ ]:
# Train dataset 불러오기

train_data = np.load("train_data2.npy")
train_label = np.load("train_label2.npy")

In [ ]:
#metric 및 loss함수로 사용할 함수 지정

def r2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def score(y_test, y_pred):
    return rmse(y_test, y_pred) / r2(y_test, y_pred)

In [ ]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
#model.add(GRU(256, input_shape=input_shape))
#model.add(GRU(32, return_sequences=False))
#model.add(Dense(4, activation = 'relu'))

model.add(layers.Conv1D(256, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(512, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Flatten())
model.add(Dense(4, activation = 'relu'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,
                                     beta_1=0.9,
                                     beta_2=0.999,
                                     epsilon=1e-07)

# loss 함수에 score함수를 지정하였으나 학습속도가 너무 느려 rmse를 활용
model.compile(optimizer=optimizer,loss = rmse, metrics=[score,'mae'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 28, 256)           19456     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 14, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 512)           393728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 512)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 64)             98368     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

In [ ]:
model.fit(train_data, train_label, epochs=100, batch_size=1024) 
# batch_size는 512를 사용하여도 생각외로 높은 성능을 보임
# 시간이 많다면 epoch를 크게 높이는 것도 방법...

Epoch 1/100
259/259 [==============================] - 9s 21ms/step - loss: 38.4507 - score: 24.9487 - mae: 34.5693
Epoch 2/100
259/259 [==============================] - 6s 21ms/step - loss: 6.7865 - score: 6.9654 - mae: 6.0678
Epoch 3/100
259/259 [==============================] - 5s 21ms/step - loss: 9.4956 - score: 9.8246 - mae: 8.8527
Epoch 4/100
259/259 [==============================] - 5s 21ms/step - loss: 7.4748 - score: 7.6204 - mae: 6.8499
Epoch 5/100
259/259 [==============================] - 5s 21ms/step - loss: 6.7634 - score: 6.8718 - mae: 6.1516
Epoch 6/100
259/259 [==============================] - 5s 21ms/step - loss: 6.0235 - score: 6.1061 - mae: 5.4148
Epoch 7/100
259/259 [==============================] - 5s 21ms/step - loss: 3.7668 - score: 3.7858 - mae: 3.2095
Epoch 8/100
259/259 [==============================] - 5s 21ms/step - loss: 3.8690 - score: 3.8906 - mae: 3.3494
Epoch 9/100
259/259 [==============================] - 5s 21ms/step - loss: 3.7403 - score: 3

259/259 [==============================] - 5s 20ms/step - loss: 1.9265 - score: 1.9290 - mae: 1.6757
Epoch 73/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9485 - score: 1.9511 - mae: 1.6970
Epoch 74/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9370 - score: 1.9396 - mae: 1.6778
Epoch 75/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9203 - score: 1.9229 - mae: 1.6681
Epoch 76/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9274 - score: 1.9300 - mae: 1.6753
Epoch 77/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9189 - score: 1.9214 - mae: 1.6803
Epoch 78/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9070 - score: 1.9096 - mae: 1.6537
Epoch 79/100
259/259 [==============================] - 5s 20ms/step - loss: 1.9341 - score: 1.9367 - mae: 1.6772
Epoch 80/100
259/259 [==============================] - 5s 20ms/step - loss: 1.8588 - score: 1.8611 -

In [ ]:
# 검증용 데이터 만들기
test_data = []

tmp = pd.read_csv(f_list[-1])
tmp = tmp.replace(" ", np.nan)
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
tmp.index = tmp.ymdhm

tmp['wl_1018662'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018662']
tmp['wl_1018680'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018680']
tmp['wl_1018683'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018683']
tmp['wl_1019630'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1019630']

tmp = tmp.reset_index(drop=True)

tmp[["wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:1][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-1][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)  
tmp[["wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]] = tmp[["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]] - pd.concat([tmp[:2][["wl_1018662", "wl_1018680", "wl_1018683","wl_1019630"]], tmp[:-2][["wl_1018662", "wl_1018680","wl_1018683","wl_1019630"]]], axis = 0).reset_index(drop=True)

tmp

<ipython-input-17-cc1cb63b7a94>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['wl_1018662'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018662']
<ipython-input-17-cc1cb63b7a94>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['wl_1018680'].iloc[-6912:] = pd.read_csv("정답_submission.csv", index_col = 'ymdhm')['wl_1018680']
<ipython-input-17-cc1cb63b7a94>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

,ymdhm,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,...,wl_1019630,fw_1019630,wl_1018662_1,wl_1018680_1,wl_1018683_1,wl_1019630_1,wl_1018662_2,wl_1018680_2,wl_1018683_2,wl_1019630_2
0,2022-05-01 00:00,0.0,0.0,0.0,25.20,739.36,218.73,25.27,124.36,81.0,...,269.3,471.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-05-01 00:10,0.0,0.0,0.0,25.20,124.48,218.73,25.27,124.48,72.0,...,266.3,438.33,1.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0
2,2022-05-01 00:20,0.0,0.0,0.0,25.20,124.20,218.73,25.27,124.20,64.0,...,264.3,417.17,-1.0,0.0,0.0,-2.0,0.0,0.0,0.0,-5.0
3,2022-05-01 00:30,0.0,0.0,0.0,25.20,124.35,218.73,25.27,124.35,58.0,...,263.3,406.79,-1.0,0.0,0.0,-1.0,-2.0,0.0,0.0,-3.0
4,2022-05-01 00:40,0.0,0.0,0.0,25.19,0.00,218.36,25.64,124.42,58.0,...,264.3,417.17,-2.0,0.0,0.0,1.0,-3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11371,2022-07-18 23:10,0.0,0.0,0.0,25.04,259.23,212.86,31.14,259.23,510.0,...,306.3,974.40,0.0,9.0,9.0,3.0,-1.0,9.0,17.0,7.0
11372,2022-07-18 23:20,0.0,0.0,0.0,25.04,260.46,212.86,31.14,260.46,492.0,...,308.3,1006.88,-1.0,8.0,5.0,2.0,-1.0,17.0,14.0,5.0
11373,2022-07-18 23:30,0.0,0.0,0.0,25.04,259.37,212.86,31.14,259.37,475.0,...,310.3,1039.90,12.0,4.0,5.0,2.0,11.0,12.0,10.0,4.0
11374,2022-07-18 23:40,0.0,0.0,0.0,25.04,259.13,212.86,31.14,259.13,458.0,...,312.3,1073.46,10.0,4.0,5.0,2.0,22.0,8.0,10.0,4.0


In [ ]:
for j in tqdm(range(len(tmp)-window)): 
    test_data.append(np.array(tmp.loc[j:j + window-1,
                                      ['rf_10184100','rf_10184110','rf_10184140',"swl", "inf", "sfw", "ecpc",
                                       "tototf", "tide_level","fw_1018662", "fw_1018680","fw_1018683", "fw_1019630",
                                       "wl_1018662", "wl_1018680", "wl_1018683","wl_1019630",
                                       "wl_1018662_1", "wl_1018680_1", "wl_1018683_1", "wl_1019630_1",
                                       "wl_1018662_2", "wl_1018680_2", "wl_1018683_2", "wl_1019630_2"]]).astype(float))

100%|██████████| 11346/11346 [00:04<00:00, 2364.58it/s]


In [ ]:
test_data = np.array(test_data)

print(test_data.shape)
print(test_data[0].shape)

(11346, 30, 25)
(30, 25)


In [ ]:
#제출 파일 만들기

pred = model.predict(test_data[-6912:])
pred = pd.DataFrame(pred)

sample_submission = pd.read_csv("sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

sample_submission.to_csv("baseline.csv", index = False)